In [26]:
pip install xgboost

     --------------------------------------- 70.9/70.9 MB 31.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [46]:
import numpy as np 
import BinOP
from sklearn.model_selection    import train_test_split
from sklearn.linear_model       import LinearRegression
from sklearn.ensemble           import RandomForestRegressor
from sklearn.metrics            import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm                import SVR
import xgboost as xgb
import tensorflow as tf
from tensorflow                 import keras

from tensorflow.keras.callbacks import EarlyStopping

## Generating data

We start by obtaining the data to train our models. For that, we generate N random data points in the intervals defined above

In [15]:
N  = 10000

M0 = 0.4 
Mf = 1.6 
T0 = 0.2 
Tf = 1.1
r0 = 0.02
rf = 0.1
v0 = 0.01 
vf = 1.0 

In [16]:
coordM = np.random.uniform(low = M0, high = Mf, size = N)
coordT = np.random.uniform(low = T0, high = Tf, size = N)
coordr = np.random.uniform(low = r0, high = rf, size = N)
coordv = np.random.uniform(low = v0, high = vf, size = N)

In [17]:
target_values = BinOP.valueBinOp(coordr, coordv, coordM, coordT, 1, 'P', 'A', 32, gamma_par = 1) 

And we split the data points generated into training and testing data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(np.column_stack((coordM, coordT, coordr, coordv)), target_values, test_size = 0.2)

##  Predictions

### Random forest

In [20]:
rf_model = RandomForestRegressor(n_estimators = 100) 
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [21]:
rf_predictions = rf_model.predict(X_test)

In [33]:
rf_mse  = mean_squared_error(y_test, rf_predictions) 
rf_rmse = np.sqrt(rf_mse)
rf_mae  = mean_absolute_error(y_test, rf_predictions)
rf_r2   = r2_score(y_test, rf_predictions) 

In [23]:
print("MSE: "  + str(RF_mse)) 
print("RMSE: " + str(RF_rmse))
print("MAE: "  + str(RF_mae))
print("R2: "   + str(RF_r2))

MSE: 3.4073683252883386e-05
RMSE: 0.005837266762182741
MAE: 0.0037968289295376633
R2: 0.9989069651588821


### Kernel methods

In [30]:
svr_model = SVR(kernel = 'rbf')

In [31]:
svr_model.fit(X_train, y_train)

SVR()

In [32]:
svr_predictions = svr_model.predict(X_test) 

In [34]:
svr_mse  = mean_squared_error(y_test, svr_predictions) 
svr_rmse = np.sqrt(svr_mse) 
svr_mae  = mean_absolute_error(y_test, svr_predictions) 
svr_r2   = r2_score(y_test, svr_predictions) 

In [35]:
print("MSE: "  + str(svr_mse))
print("RMSE: " + str(svr_rmse))
print("MAE: "  + str(svr_mae))
print("R2: "   + str(svr_r2))

MSE: 0.0011876628820207578
RMSE: 0.0344624851399425
MAE: 0.0274330120492494
R2: 0.9622948457413356


### Neural Networks

In [47]:
nn_model = keras.Sequential([
    keras.layers.Dense(64, activation = 'relu', input_shape = (4,)),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1) 
])

In [51]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
nn_model.compile(optimizer = 'adam', loss = 'mean_squared_error') 

In [52]:
history = nn_model.fit(X_train, y_train, epochs = 100, batch_size = 16, validation_data = (X_test, y_test))

Epoch 1/100
500/500 [==============================] - 3s 5ms/step - loss: 5.4146e-05 - val_loss: 2.5363e-05
Epoch 2/100
500/500 [==============================] - 2s 4ms/step - loss: 4.2286e-05 - val_loss: 3.5958e-05
Epoch 3/100
500/500 [==============================] - 2s 4ms/step - loss: 3.5280e-05 - val_loss: 2.2933e-05
Epoch 4/100
500/500 [==============================] - 2s 4ms/step - loss: 3.3425e-05 - val_loss: 2.8516e-05
Epoch 5/100
500/500 [==============================] - 2s 4ms/step - loss: 3.2541e-05 - val_loss: 3.0144e-05
Epoch 6/100
500/500 [==============================] - 2s 4ms/step - loss: 2.2778e-05 - val_loss: 1.9851e-05
Epoch 7/100
500/500 [==============================] - 2s 4ms/step - loss: 2.6070e-05 - val_loss: 2.5988e-05
Epoch 8/100
500/500 [==============================] - 2s 4ms/step - loss: 2.3034e-05 - val_loss: 2.0204e-05
Epoch 9/100
500/500 [==============================] - 2s 4ms/step - loss: 2.3565e-05 - val_loss: 2.3484e-05
Epoch 10/100
500/50

500/500 [==============================] - 2s 4ms/step - loss: 6.4201e-06 - val_loss: 6.4750e-06
Epoch 76/100
500/500 [==============================] - 2s 4ms/step - loss: 6.3860e-06 - val_loss: 5.3147e-06
Epoch 77/100
500/500 [==============================] - 2s 4ms/step - loss: 7.5792e-06 - val_loss: 5.6106e-06
Epoch 78/100
500/500 [==============================] - 2s 4ms/step - loss: 5.8895e-06 - val_loss: 3.7783e-06
Epoch 79/100
500/500 [==============================] - 2s 4ms/step - loss: 4.6369e-06 - val_loss: 4.9807e-06
Epoch 80/100
500/500 [==============================] - 2s 4ms/step - loss: 5.6686e-06 - val_loss: 5.2940e-06
Epoch 81/100
500/500 [==============================] - 2s 4ms/step - loss: 6.4398e-06 - val_loss: 3.7139e-06
Epoch 82/100
500/500 [==============================] - 2s 4ms/step - loss: 7.0889e-06 - val_loss: 8.9315e-06
Epoch 83/100
500/500 [==============================] - 2s 4ms/step - loss: 5.7190e-06 - val_loss: 2.9875e-06
Epoch 84/100
500/500 [=

In [55]:
input_data = np.array([1.1, 0.5, 0.06, 0.3]) 
input_data = input_data.reshape(1,-1)
nn_model.predict(input_data)

1/1 [==============================] - 0s 86ms/step


array([[0.03757279]], dtype=float32)

In [56]:
 BinOP.valueBinOp(0.06, 0.3, 1.1, 0.5, 1, 'P', 'A', 32, gamma_par = 1) 

0.038962106005657464